# 草稿を作成する
リサーチフローの環境を利用して論文の草稿や論文に使用する図表の作成を行うタスクです。

## 草稿を作成する
リサーチフロー上でLaTex（文章作成ツール）を使用して論文の草稿を作成することが可能です。<br>
LaTexを使用する場合は以下の説明に従って草稿の作成、および保存を行ってください。<br>
リサーチフロー外で作成する場合は草稿の作成後、「フォルダを表示する」セルを実行して表示されるフォルダに直接アップロードしてください。

### リサーチフロー上での草稿の作成方法
1. 「フォルダを表示する」セルを実行することで草稿を作成するフォルダが表示されます。<br>
2. 画像の説明に従って草稿を作成してください。<br>
<br>
![ファイル作成](./images/RF004005_create_file.png)<br>
![ファイル改名](./images/RF004005_rename_file.png)<br>
<span style="color:red">※ 草稿の編集後は必ず保存してからファイルを閉じてください</span>　

In [ ]:
# コピーしたデータを表示する
import os
import panel as pn
from IPython.display import display
from IPython.core.display import Javascript
from library.utils.access import open_data_folder
from library.task_director import TaskDirector

notebook_name = 'create_draft.ipynb'

def access_draft_folder(working_path: str):
    """草稿フォルダを表示するメソッドです。"""

    folder_name = 'draft'
    button = open_data_folder(os.path.abspath('__file__'), folder_name)

    pn.extension()
    display(button)
    display(Javascript('IPython.notebook.save_checkpoint();'))
    task_director = TaskDirector(working_path, notebook_name)
    task_director.done_task()

access_draft_folder(os.path.abspath('__file__'))

3. 作成したtex形式の草稿をPDFとして出力する<br>
terminalで以下のコマンドを実行することでtex形式のファイルを同じフォルダ内にPDFで出力することができます。<br>
PDFファイルはtexファイルと同じ名前で、同名のPDFファイルが存在していた場合は上書きします。<br>
pdflatex {対象のファイルまでのパス}<br>
<br>
以下のセルにファイル名を入力して実行することでterminalを使用せずにPDF化することも可能です。

In [ ]:
# texファイルをPDFとして出力する
import os
from library.utils.setting import get_data_dir

file_name ="" #　ここにＰＤＦ化したいファイルを入力する(.texまで含む)

dir_name = os.path.join(get_data_dir(os.path.abspath('__file__')), 'draft')
full_path = os.path.join(dir_name, file_name)

# PDF化するコマンド
!pdflatex -output-directory={dir_name} {full_path}

## 図表を作成する
Notebookを使用してリサーチフロー上で図表の作成が可能です。<br>
新しく図表を作成する場合は「図表を新たに作成する」ボタンを押下することで図表作成用のノートブックが作成されます。<br>
<br>
既に作成した図表の編集を行いたい場合は以下の手順に従ってください。<br>
①「図表を編集する」セルを実行して、図表と図表生成用のノートブックが保存されているフォルダを表示する。<br>
② 編集したい図表、またはノートブックを開いて編集する。<br>
<br>
また、リサーチフローを用いずに図表を作成した場合は以下の手順でアップロードを行ってください。<br>
①「図表を編集する」セルを実行して、図表と図表生成用のノートブックが保存されているフォルダを表示する。<br>
②「upload」ボタンから作成した図表をアップロードする。<br>

### 1. 図表を新たに作成する

In [ ]:
# 図表を新たに作成する
import os
import shutil

from IPython.core.display import Javascript
import panel as pn

from library.task_director import TaskDirector
from library.utils.access import open_data_file
from library.utils.config import message as msg_config
from library.utils.setting import get_data_dir
from library.utils.widgets import Button, MessageBox


notebook_name = 'create_draft.ipynb'
template_notebook_path = '/home/jovyan/data_governance/base/task/writing/figure_template.ipynb'


class CreateFigureNoteBook(TaskDirector):
    """図表用ノートブックを作成するクラスです。

    Attributes:
        instance:
            working_path(str): 実行Notebookファイルパス
            output_message(MessageBox): メッセージ出力用のボックス
            form_section(pn.WidgetBox): ボタン等出力用のボックス
            text_input(pn.widgets.TextInput): ノートブック名を入力するためのフォーム
            generate_button_title(str): ノートブック作成用ボタンのタイトル文
            generte_button(Botton): ノートブック作成時に押下するボタン

    """

    def __init__(self, working_path: str):
        """CreateFigureNoteBookクラスのコンストラクタです。

        Args:
            working_path (str): 実行Notebookファイルパス

        """
        self.working_path = working_path
        super().__init__(self.working_path,  notebook_name)

        # メッセージ出力用のボックス
        self.output_message = MessageBox()
        self.output_message.width = 900
        # ボタン等表示用のボックス
        self.form_section = pn.WidgetBox()

    @TaskDirector.task_cell("3")
    def generate_name_form(self):
        """図表用ノートブックの名前を入力し、作成するためのフォームを作成するメソッドです。"""
        # 名前入力用フォームの設定
        self.text_input = pn.widgets.TextInput(name=msg_config.get(
            'create_draft', 'create_notebook'), width=300, height=50)
        # 作成時に押下するボタンの設定
        self.generate_button_title = msg_config.get(
            'create_draft', 'create_notebook')
        self.generate_button = Button(width=500)
        self.generate_button.set_looks_init(self.generate_button_title)
        self.generate_button.on_click(self.generate)

        # 表示する
        pn.extension()
        self.form_section.append(self.text_input)
        self.form_section.append(self.generate_button)
        display(self.form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))

    @TaskDirector.callback_form('図表用ノートブックを作成する')
    def generate(self, event):
        """ボタン押下時に図表用ノートブックを作成し、表示するためのボタンを出力するメソッドです。"""

        # 値が入力されていない場合
        if not self.text_input.value:
            self.output_message.update_warning(
                msg_config.get('create_draft', 'name_no_value'))
            self.form_section.append(self.output_message)
            return

        full_file_name = self.text_input.value + ".ipynb"
        dir_name = os.path.join(get_data_dir(self.working_path), 'figure')
        output_file_path = os.path.join(dir_name, full_file_name)

        # 入力した値のファイルが既に存在する場合
        if os.path.exists(os.path.join(dir_name, full_file_name)):
            self.output_message.update_warning(
                msg_config.get('create_draft', 'file_existed_warning'))
            return

        # テンプレートをコピーして新しいノートブックを作成
        shutil.copy(template_notebook_path, output_file_path)

        self.output_message.update_success(
            msg_config.get("create_draft", "success_create"))
        button = open_data_file(
            self.working_path, os.path.join('figure', full_file_name))

        # 表示を切り替え
        self.form_section.clear()
        self.form_section.append(self.output_message)
        self.form_section.append(button)
        display(Javascript('IPython.notebook.save_checkpoint();'))


CreateFigureNoteBook(working_path=os.path.abspath('__file__')).generate_name_form()

### 2. 図表を編集する
作成済みの図表を編集したい場合は「図表フォルダを表示する」セルを実行してください。<br>
「データフォルダを表示する」ボタンを押下することで図表および図表用ノートブックが保存されているフォルダを表示します。<br>
リサーチフロー外から図表データをアップロードしたい場合は以下の図の説明に従って実行してください。<br>
![画像をアップロードする](./image/RF004005_figure_upload.png)

In [ ]:
# 図表フォルダを表示する
import os
import panel as pn
from IPython.display import display
from IPython.core.display import Javascript
from library.utils.access import open_data_folder

folder_name = 'figure'

button = open_data_folder(os.path.abspath('__file__'), folder_name)

pn.extension()
display(button)
display(Javascript('IPython.notebook.save_checkpoint();'))

## 執筆における注意事項を確認する
論文の執筆を行う際は以下の点に注意してください。<br>
<br>
・投稿先のテーマや目的に沿った記述を行う。<br>
・規定に従った書き方をする。<br>
・誤字脱字がないようにする。<br>
・参考文献が適切か確認する。<br>
・論文の構成を論理的かつ明確にする。<br>
・データの信頼性と再現性を確保する。<br>
・図表やグラフを適切に活用する。<br>
・研究の独自性や新規性を強調する。<br>
・倫理的な配慮を行う。<br>
など

## GakuNin RDMに保存する

In [ ]:
# GakuNin RDMに保存する
from IPython.core.display import Javascript
from IPython.display import display

import panel as pn

from library.task_director import TaskDirector
from library.utils.setting import get_data_dir

script_file_name = 'save_data'
notebook_name = script_file_name+'.ipynb'


class DataSaver(TaskDirector):
    """GRDMに保存するクラスです。

    Attributes:
        instance:
            _abs_root_path (str): 絶対パス
            save_form_box(pn.WidgetBox):フォームを格納する。
            save_msg_output(Message):ユーザーに提示するメッセージを格納する。
    """

    def __init__(self, working_path: str) -> None:
        """DataSaver コンストラクタメソッドです。

        Args:
            working_path (str): 実行Notebookファイルパス
        """
        self.working_path = working_path
        super().__init__(self.working_path, notebook_name)

    @TaskDirector.task_cell("1")
    def generate_form_section(self):
        """取得したデータを表示するメソッドです。"""
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task()

        # フォーム定義
        data_dir = get_data_dir(self.working_path)
        source = [os.path.join(data_dir, 'argument_data'), os.path.join(data_dir, 'draft'), os.path.join(data_dir, 'figure')]
        self.define_save_form(source)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self.save_form_box)
        form_section.append(self.save_msg_output)
        display(form_section)
        display(Javascript('IPython.notebook.save_checkpoint();'))


DataSaver(working_path=os.path.abspath('__file__')).generate_form_section()

## サブフローメニューを表示する

In [ ]:
# サブフローメニューを表示する
import os

from library.task_director import TaskDirector

script_file_name = "make_research_data_management_plan"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()
